Importamos las diferentes librerías que nos van a permitir trabajar con LLM, y esta es principalmente LangChain ... (descripción de LangChain)

In [12]:
import os
import dotenv

from typing import List

from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document


import numpy as np  
import pandas as pd

from surprise import ( 
  Dataset,
  Reader,
  accuracy, 
  SVD,
  AlgoBase,
  BaselineOnly,
  NormalPredictor,
  KNNBasic,
  KNNWithMeans,
  KNNBaseline,
  KNNWithZScore,
  SVDpp,
  NMF,
  SlopeOne,
  CoClustering,
  accuracy
) 

from surprise.model_selection import (
  train_test_split
)


Estos métodos nos van a permitir trabajar con los modelos, pero antes es necesario las credenciales de Google API para poder usarlos. 

> El modelo que estaremos usando es `gemini-1.5-pro`

In [5]:
def load_environment () -> None:
  """
  Carga las variables de entorno necesarias para el funcionamiento del programa

  Especificamente buscar las variables de entorno siguientes:
  - `google_api_key` : clave de API de Google  
  """
  
  dotenv.load_dotenv()
  os.environ.setdefault ( 'google_api_key', os.getenv('google_api_key') )

def get_model() -> GoogleGenerativeAI:
  """
  Inicializa y devuelve una instancia de GoogleGenerativeAI
  con configuraciones predeterminadas 

  Contiene una funcion para cargar las credenciales de Google API 
  desde el entorno y crea una instancia de GoogleGenerativeAI
  usando el modelo 'gemini-1.5-pro'

  Args:

  Returns:
      GoogleGenerativeAI: instancia preconfigurada de GoogleGenerativeAI
  """

  load_environment()
  model = GoogleGenerativeAI(
    model='models/gemini-1.5-pro-latest',
    temperature=0.5
  )
  return model

def get_embedding() -> GoogleGenerativeAIEmbeddings:
  """
  Inicializa y devuelve una instancia de `GoogleGenerativeAIEmbeddings`

  Esta funcion carga las credenciales de Google API desde el entorno y crea una instancia de `GoogleGenerativeAIEmbeddings`

  Returns:
      GoogleGenerativeAIEmbeddings: instancia preconfigurada del embedding model 
  """

  load_environment()
  embedding = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001'
  )
  return embedding 

# 

Ahora definimos una función sencilla para poder probar el modelo de lenguaje

In [3]:
def prompt_template_QA(question: str, k: int, model: GoogleGenerativeAI) -> str:
  """
  Este metodo construye un template de chat que incluye instrucciones claras para el modelo de IA sobre como responder 
  a una pregunta especifica y sugerir posibles preguntas relacionadas. 

  Utiliza parametro `k` para especificar la cantidad de recomendaciones de preguntas debe incluir en su respuesta

  Args:
      question (str): la pregunta especifica que se desea que el modelo responda 
      k (int): cantidad de recomendaciones de preguntas relacionadas que se deben incluir en la respuesta 
      model (GoogleGenerativeAI): instancia del modelo de IA utilizado para generar respuesta

  Returns:
      result (str): respuesta generada por el modelo, incluyendo tanto la respuesta directa a la pregunta como las recomendaciones de preguntas relacionadas
      
  """

  prompt = ChatPromptTemplate.from_template(
    """ 
    Se lo mas simple posible para responder la siguiente pregunta 
    y da algunas recomendaciones a preguntas que se parezcan al tema de la pregunta

    Solo devuelve la respuesta. Seguido de las preguntas. Ejemplo:
    
    Answer

    Posibles preguntas:
    - Pregunta sugerida 1
    - Pregunta sugerida 2
    - Pregunta sugerida 3  

    El numero de preguntas que sugieres debe estar fijado al siguiente numero:
    Numero de recomendaciones: {k}

    Q: {question}
    A: 
    """
  )
  
  chain = prompt | model 
  result = chain.invoke(
    {
      "question": question,
      "k": k
    })
  return result


In [6]:
llm_model = get_model ( )
llm_model

GoogleGenerativeAI(model='models/gemini-1.5-pro-latest', temperature=0.5, client=genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
))

In [11]:
print ( prompt_template_QA ( question='Que es un sistema de recomendacion', k=5, model=llm_model ) )

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Un sistema de recomendación sugiere elementos a los usuarios. Los elementos pueden ser cualquier cosa, como películas, libros, productos, etc. 

Posibles preguntas:
- ¿Cómo funcionan los sistemas de recomendación?
- ¿Cuáles son los diferentes tipos de sistemas de recomendación?
- ¿Cuáles son las ventajas de usar un sistema de recomendación?
- ¿Cuáles son algunos ejemplos de sistemas de recomendación en el mundo real?
- ¿Cuáles son algunos de los desafíos en la construcción de sistemas de recomendación? 



Ahora vamos a definir un punto importante para formar un *Sistema de Recomendación usando LLM*: la característica de Chat History

In [13]:
class DataLoader: 
  
  def __init__(self, data_path: str, item_path: str, user_path: str) -> None:
    
    current = os.getcwd () [ 0 : os.getcwd ().rfind( '\\' ) ]
    self.DATA_PATH = current + data_path
    self.ITEM_PATH = current + item_path
    self.USER_PATH = current + user_path

    self.data_set = self.load_set ( 'DATA' )
    self.item_set = self.load_set ( 'ITEM' )
    self.user_set = self.load_set ( 'USER' )

  def load_set (self, name: str ) -> pd.DataFrame:

    if name == 'DATA':
      columns = [ 'userID', 'itemID', 'rating', 'timestamp' ]
      df = pd.read_csv ( 
        self.DATA_PATH, 
        names=columns, 
        sep='\t', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'timestamp' ] )
      return df
  
    if name == 'USER':
      columns = [ 'userID', 'age', 'gender', 'occupation', 'zipCode' ]
      df = pd.read_csv ( 
        self.USER_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'zipCode' ] )
      return df
  
    if name == 'ITEM':
      columns = [ 
        'itemID', 
        'name', 
        'releaseDate', 
        'videoReleaseDate', 
        'IMDbURL', 
        'gender_unknown', 
        'gender_action', 
        'gender_adventure', 
        'gender_animation', 
        'gender_children', 
        'gender_comedy',
        'gender_crime',
        'gender_documentary',
        'gender_drama',
        'gender_fantasy',
        'gender_film_noir',
        'gender_horror',
        'gender_musical',
        'gender_mystery',
        'gender_romance',
        'gender_scifi',
        'gender_thriller',
        'gender_war',
        'gender_western',
      ]
      df = pd.read_csv ( 
        self.ITEM_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'videoReleaseDate', 'IMDbURL' ] )
      return df

  def load_dataset ( self ) -> Dataset:
    reader = Reader ( rating_scale= ( 1,5 ) )
    data = Dataset.load_from_df ( self.data_set [ [ 'userID', 'itemID', 'rating' ] ], reader )
    return data

  def get_user_by_id ( self, id: int ):
    info = self.user_set.loc [ self.user_set[ 'userID' ] == id ]
    return info[ [ 'userID', 'age', 'gender', 'occupation' ] ].iloc[0].to_dict()

  def get_item_by_id ( self, id: int ):
    info = self.item_set.loc [ self.item_set[ 'itemID' ] == id ]
    return info[ [ 
      'itemID', 
      'name', 
      'releaseDate', 
      'gender_unknown', 
      'gender_action', 
      'gender_adventure', 
      'gender_animation', 
      'gender_children', 
      'gender_comedy',
      'gender_crime',
      'gender_documentary',
      'gender_drama',
      'gender_fantasy',
      'gender_film_noir',
      'gender_horror',
      'gender_musical',
      'gender_mystery',
      'gender_romance',
      'gender_scifi',
      'gender_thriller',
      'gender_war',
      'gender_western', ] ].iloc[0].to_dict()

  def get_rating_by_ids ( self, user_id: int, item_id: int ):
    try:
      rating = self.data_set.loc [ self.data_set[ 'userID' ] == user_id ].loc [ self.data_set[ 'itemID' ] == item_id ]
      return ( rating.iloc[0]['rating'], True )
    except:
      # Failed to retrieve the rating
      return ( -1, False )

  def get_ratings_by_name_id ( self, column_name: str, id: int ):
    filtered_data = self.data_set.loc [ self.data_set[ column_name ] == id ]
    return filtered_data [ [ 'userID', 'itemID', 'rating' ] ]
  


DATA_PATH = '\\dataset\\data.csv'
ITEM_PATH = '\\dataset\\item.csv'
USER_PATH = '\\dataset\\user.csv'

loader = DataLoader( 
  data_path=DATA_PATH,
  item_path=ITEM_PATH,
  user_path=USER_PATH 
)

In [14]:
loader.data_set

,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [15]:
loader.item_set

,itemID,name,releaseDate,gender_unknown,gender_action,gender_adventure,gender_animation,gender_children,gender_comedy,gender_crime,...,gender_fantasy,gender_film_noir,gender_horror,gender_musical,gender_mystery,gender_romance,gender_scifi,gender_thriller,gender_war,gender_western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
loader.user_set

,userID,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
...,...,...,...,...
938,939,26,F,student
939,940,32,M,administrator
940,941,20,M,student
941,942,48,F,librarian


In [ ]:
# Funciones utiles para esta caracteristica



In [ ]:

model = get_model()
embedding = get_embedding()

current = os.getcwd()
doc_dir = '\\database\\doc'
faiss_dir = '\\database\\faiss'

DATA_PATH = current + doc_dir
FAISS_PATH = current + faiss_dir

dir = os.listdir(DATA_PATH)



def search_docs() -> list[str]:
  """
  Busca todos los documentos disponibles en el directorio especificado y retorna sus rutas completas

  Este metodo recorre el directorio establecido en la variable global 'DATA_PATH', buscando todos los archivos presentes y retornando sus rutas completas como una lista de cadenas
  
  Nota: Las rutas devueltas son relativas al directorio actual del proyecto

  Returns:
      list[str]: una lista de cadenas donde cada elemento es la ruta completa de un documento encontrado en el directorio especificado
  """
  docs_dir = os.listdir(DATA_PATH)
  docs_dir = [DATA_PATH + '\\' + item for item in docs_dir]
  return docs_dir



def load_contents(docs_dir: list[str]) -> list[str]:
  """
  Lee y retorna el contenido de todos los documentos especificados por sus rutas.

  Este metodo itera sobre cada ruta del documento proporcionada en la lista 'docs_dir', lee el contenido de cada uno de estos archivos y los agrega a una lista, la cual luego retorna

  Nota: Los archivos deben estar en formato de texto plano para poder ser leidos correctamente por este metodo

  Args:
      docs_dir (list[str]): Lista de rutas de directorios completas a los documentos cuyo contenido se desea leer

  Returns:
      list[str]: Lista de cadenas donde cada elemento es el contenido leido de un documento correspondiente a las rutas proporcionadas
  """



  def load_content(doc_dir: str):
    content = ''
    with open(doc_dir, 'r', encoding='ISO-8859-1') as file:
      content = file.read()
    return content  
  
  docs_content = [load_content(doc_dir=doc_dir) for doc_dir in docs_dir]
  return docs_content 



def chunkenizer(content: str) -> List[Document]:
  """
  Este metodo se encarga de separar en chunks el contenido de un documento. Esta fragmentacion la hace usando un metodo recursivo llamado `RecursiveCharacterTextSplitter` con valores predeterminados

  Args:
      content (str): contenido de un documento correspondiente

  Returns:
      List[Document]: lista de chunks del documento de entrada 
  """

  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap = 204,
    length_function = len
  )
  chunks = text_splitter.create_documents([content])
  return chunks



def create_chunks() -> List[Document]:
  """
  Divide el contenido de todos los documentos encontrados en chunks. 

  Este metodo primero busca todos los documentos disponibles en el directorio especificado, 
  luego lee y retorna el contenido de estos documentos. Posteriormente, divide el contenido de cada documento
  en chunks utilizando un metodo especifico de division de texto, y retorna una lista de objetos `Document`, 
  donde cada objeto representa un chunk del contenido original

  Returns:
      List[Document]: Una lista de objetos Document, donde cada objeto representa un chunk del contenido de un documento
  """

  docs_dir = search_docs()
  docs_content = load_contents(docs_dir)
  chunks: List[Document] = []
  for doc_content in docs_content:
    
    document = Document(page_content=doc_content)
    chunks.append(document)
    
    """ 
    # CHUNK THE DOCUMENTS
    for item in chunkenizer(doc_content):
      chunks.append(item)
    """
    
  return chunks



class Faiss_Vectorstore:



  def __init__(self, load: bool = False) -> None:
    """
    Inicializa una instancia de `FAISS_VECTORSTORE` para manejar y buscar en una base de datos vectorial de FAISS

    Este constructor permite opcionalmente cargar la base de datos vectorial existente si se pasa `True` al parametro `load`. 
    Si `load` es `False` (valor predeterminado), se crea una nueva base de datos vectorial con los documentos proporcionados

    Args:
        load (bool, optional): Indica si se debe cargar la base de datos vectorial. Defaults to False.
    """
    if load: 
      self.__vs = FAISS.load_local(
        folder_path=FAISS_PATH, 
        embeddings=embedding,
        allow_dangerous_deserialization=True)
    else:
      chunks = create_chunks()
      
      """ 
      ERROR POR LA CANTIDAD DE ELEMENTOS DE CHUNKS (debe ser por eso)
      self.__vs = FAISS.from_documents(
        documents=chunks,
        embedding=embedding
      )
      """

      self.__vs = FAISS.from_documents(
        documents=chunks[0:100], 
        embedding=embedding)

      i = 101
      while True:
        if i >= len(chunks):
          break
        if i + 99 >= len(chunks):
          extension = FAISS.from_documents(
            documents=chunks[i:len(chunks)],
            embedding=embedding
          )
          self.__vs.merge_from(extension)
          break
        extension = FAISS.from_documents(
          documents=chunks[i:i+99],
          embedding=embedding
        )
        i=i+100
      self.__vs.save_local(FAISS_PATH)



  def similarity_search(self, query: str, k: int = 3) -> list[str]:
    """
    Realiza una busqueda de similaridad en la base de datos vectorial utilizando una consulta 

    Este metodo busca en los documentos, los similares a la consulta proporcionada, utilizan el numero de resultados `k`

    Args:
        query (str): la consulta de texto para realizar la busqueda de similitud
        k (int, optional): numero de resultados similares a retornar. Defaults to 3.

    Returns:
        list[str]: una lista de cadenas donde cada elemento es el contenido de un documento encontrado que es similar a la consulta
    """
    if not k > 0:
      raise Exception("k no puede ser negativo ni 0")
    results = self.__vs.similarity_search(query=query, k=k)
    results_content = [result.page_content for result in results]
    return results_content


In [ ]:
class ChatHistory:
  
  def __init__(self, with_vectorstore: bool = True) -> None:
    self.model: GoogleGenerativeAI = get_model()
    self.embedding: GoogleGenerativeAIEmbeddings = get_embedding()
    self.chat: list = []
    self.prompt = """
    Eres un asistente, capaz de responder detalladamente las respuestas que se te hagan
    Tambien debes tener conocimiento sobre la conversacion que tengas con el usuario
    """
    if with_vectorstore: self.vectorstore = Faiss_Vectorstore(load=True)  



  def make_chain(self) -> None:
    self.prompt = ChatPromptTemplate.from_messages([
      ('system', f'{self.prompt}'),
      MessagesPlaceholder(variable_name='chat'),
      ('human', '{input}')
    ])
    self.chain = self.prompt | self.model



  def clean_history(self) -> None:
    self.chat: list = []



  def send_processed_query(self, query: str) -> str:
    response = self.chain.invoke({'input':query, 'chat':self.chat})
    self.chat.append( HumanMessage(content=query) )
    self.chat.append( AIMessage(content=response) )

    return response


Ahora vamos a ejecutar un poco de pruebas sobre el LLM con Chat History para ver sus capacidad para recordar y aprender de la conversación que se tenga con este.

In [ ]:
# COMPROBAR QUE SE CREE LA BASE DE DATOS VECTORIAL CORRECTAMENTE 

def test__create_vectorstore():
  Faiss_Vectorstore()


In [ ]:
# COMPROBAR QUE LA BUSQUEDA POR SIMILITUD QUE TRAE LA CLASE `FAISS_VECTORE` FUNCIONE

def testing_load_and_query_something():
  vs = Faiss_Vectorstore(load=True)
  result = vs.similarity_search(query='que es la apologetica', k=10)
  for item in result:
    print(item)